# NestedPKE

## Introduction

This is a public key encryption scheme where the message is encrypted under one PKE scheme, then that ciphertext is encrypted under a second PKE scheme; the resulting scheme is secure if either of the two PKE schemes is. 

This example actually contains two proofs of IND-CPA security:

- The first proof assumes security of the inner PKE scheme. This proof is extremely simple and consists of a single game hop based on the indistinguishability of the inner PKE scheme. This is an example of an *indistinguishability proof step* and shows how to use a reduction to an indistinguishability property to relate two games.
- The second proof assumes security of the outer PKE scheme. This proof introduces *rewriting proof steps*, which are needed to make use of certain facts known to the cryptographer but not encoded in the formalism (e.g., that two messages of the same length encrypt to two ciphertexts of the same length

## Defining NestedPKE

First we will give the specification of the NestedPKE scheme, constructed from two generic PKE schemes.

To get started, we have to import the other Python packages that our example depends on. The imports from `typing` are used in the type annotations.

In [1]:
from typing import cast, Tuple, Union

from gamehop.primitives import Crypto, PKE
from gamehop.primitives.PKE import PKEScheme
import gamehop.inlining
import gamehop.verification

Next we declare the two PKE schemes that we will generically use to build NestedPKE.

In [2]:
class PKE1(PKEScheme): pass
class PKE2(PKEScheme): pass

Now we give the main definition of the NestedPKE scheme.

As for all public key encryption schemes (which are descendents of the `gamehop.primitives.pke.PKEScheme` class) we have to define classes representing public keys, secret keys, ciphertexts, and messages, as well as the three main operations: key generation, encryption, and decryption.

For the public key and secret key, we give explicit classes that build a compound public key / secret key from the two public keys / secret keys of the underlying PKE schemes.  For the ciphertext and message, we don't give explicit constructors. Instead, in the encryption and decryption procedures, we will cast to the required type; this models treating a PKE1 ciphertext as a PKE2 message, and treating a PKE2 ciphertext as a NestedPKE ciphertext.

In [3]:
class NestedPKE(PKEScheme):
    
    class PublicKey(PKEScheme.PublicKey):
        def __init__(self, pk1: PKE1.PublicKey, pk2: PKE2.PublicKey):
            self.pk1 = pk1
            self.pk2 = pk2
            
    class SecretKey(PKEScheme.SecretKey):
        def __init__(self, sk1: PKE1.SecretKey, sk2: PKE2.SecretKey):
            self.sk1 = sk1
            self.sk2 = sk2
            
    class Ciphertext(PKEScheme.Ciphertext): pass

    class Message(PKEScheme.Message): pass

    @staticmethod
    def KeyGen() -> Tuple[PublicKey, SecretKey]:
        (pk1, sk1) = PKE1.KeyGen()
        (pk2, sk2) = PKE2.KeyGen()
        pkprime = NestedPKE.PublicKey(pk1, pk2)
        skprime = NestedPKE.SecretKey(sk1, sk2)
        return (pkprime, skprime)
    
    @staticmethod
    def Encrypt(pk: PublicKey, msg: Message) -> Ciphertext: # type: ignore[override]
        ct1 = PKE1.Encrypt(pk.pk1, msg)
        pt2 = cast(PKE2.Message, ct1)
        ct2 = PKE2.Encrypt(pk.pk2, pt2)
        ctprime = cast(NestedPKE.Ciphertext, ct2)
        return ctprime
    
    @staticmethod
    def Decrypt(sk: SecretKey, ct: Ciphertext) -> Union[Crypto.Reject, Message]: # type: ignore[override]
        pt2 = PKE2.Decrypt(sk.sk2, ct)
        if pt2 == Crypto.Reject:
            r: Union[Crypto.Reject, NestedPKE.Message] = Crypto.Reject()
        else:
            ct1 = cast(PKE1.Ciphertext, pt2)
            pt1 = PKE1.Decrypt(sk.sk1, ct1)
            r = cast(NestedPKE.Message, pt1)
        return r


## NestedPKE is IND-CPA-secure if PKE1 is IND-CPA-secure

Once the proof engine is updated to work with games as classes, we can complete this example.  For now I would like to just print out some of the canonicalizations, such as using the following code block.

However, this code block will fail, because apparently `inspect.getsource` doesn't work for things defined within a Jupyter notebook file.  This is really sucky for us.  Lots of people have complained about it, for years, with no progress; there are some hacky solutions, so perhaps we could try modifying our `get_class_def` / `get_function_def` code to use some of those hacks.  It sucks but I think this Jupyter notebook idea is very nice so maybe it's worth trying to hack in.

- https://stackoverflow.com/questions/51566497/getting-the-source-of-an-object-defined-in-a-jupyter-notebook
- https://github.com/ipython/ipython/issues/11249
- https://bugs.python.org/issue33826

In [4]:
g0 = gamehop.inlining.inline_scheme_into_game(NestedPKE, PKE.INDCPA.get_left())
print(g0)
print(gamehop.verification.canonicalize_game(g0))

TypeError: <class '__main__.NestedPKE'> is a built-in class